In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0, "/notebooks/")
from fastai.imports import *

In [3]:
from fastai.core import V, VV_, to_np
import torch.nn.functional as F
import torch.nn as nn
from torch import zeros
import torch

n_hidden = 512
def text_model(**kwargs):
	model = CharSeqStatefulLSTM(vocab_size=55, n_fac=42, bs=64, nl=2)
	return model

class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [4]:
file_path = "/notebooks/data/nietzsche/nietzsche_cyc_4"

In [5]:
state_dict = torch.load(file_path, map_location=lambda storage, loc: storage)

In [11]:
model = text_model()

In [12]:
model.load_state_dict(state_dict)

In [ ]:
model